In [90]:
import pandas as pd 
import jieba
import jieba.analyse

df = pd.read_csv("csv_ETL//articles_of_ Peugeot.csv").astype(str)
df2 = df.loc[:,['article_sub','main_content','all_replies']]

In [95]:
car_type =[
    "1007",
    "106",
    "107",
    "2008",
    "205",
    "206",
    "207",
    "208",
    "3008",
    "301",
    "306",
    "307",
    "308",
    "406",
    "407",
    "5008",
    "508 sw",
    "607",
    "807",
    "taveller"

]



In [97]:
final_df = pd.DataFrame(columns=['content', 'type'])
jieba.enable_paddle()

for x in car_type:
    df_1 = df2.loc[df2.article_sub.str.lower().str.contains(x).fillna(False)]
    df_2 = df2.loc[df2.main_content.str.lower().str.contains(x).fillna(False)]
    df_3 = pd.concat([df_1, df_2],ignore_index=True)
    df_3 = df_3.drop_duplicates()
    df_3['all'] = df_3['article_sub'] + df_3['main_content'] + df_3['all_replies']
    df_4 = df_3.loc[:,['all']]
    d = df_4['all'].tolist()
    if len(d) >1000:
        jieba.load_userdict('user_dict.txt')
        jieba.analyse.set_stop_words('stop_words.txt')
        with open('stop_words.txt','r',encoding='utf-8') as f:
            stop_words = f.readlines()
        stop_words = [stop_word.rstrip() for stop_word in stop_words]
        seg = []
        for i in range(len(d)):
            cut_list = list( jieba.cut(d[i], cut_all = False) )
            cut_list = [w for w in cut_list if w not in stop_words]
            seg.append([' '.join([ item for item in cut_list if len(item) > 1 ])])
        seg = [ s for s in seg if len(s) > 0] 

        # 斷詞結果存檔
        segSaveFile = "Peugeot//"+x+'Done.txt'
        with open(segSaveFile, 'wb') as saveFile:
            for i in range(len(seg)):
                words = seg[i][0].encode('utf-8')
                words = words.strip()
                if len(words) > 0:
                    #saveFile.write(seg[i][0].encode('utf-8'))
                    saveFile.write(words)
                    saveFile.write('\n'.encode())

        e = pd.DataFrame(seg, columns=['content'])
        f = pd.DataFrame(['Peugeot_' + x] * e.shape[0], columns=['type'])
        g = pd.concat([e,f], axis=1)
        final_df = pd.concat([final_df, g], ignore_index = 1)
    else:
        continue


Paddle enabled successfully......


In [98]:
final_df

,content,type
0,國內 新訊 Peugeot 推出 2008 308 特仕車 搭載 1.5 BlueHDi 動...,Peugeot_308
1,308 車門 刮傷 2018 308 這裡 刮傷 經驗 解決 2018 308 這裡 刮傷 ...,Peugeot_308
2,308 SW 價位 改款 本對 獅子 路過 晃晃 308SW 實車 試乘 燒到 社團 爬文 ...,Peugeot_308
3,308SW EA 八速 有無 引進 台灣 台灣 308SW 六速 自排 今年 新款 八速 自...,Peugeot_308
4,308 T9 18 鋁框 一問 車是 308T9 想改 18 鋁框 台灣市場 符合 T9 規...,Peugeot_308
...,...,...
1257,PEUGEOT 六月 領牌數 PEUGEOT 六月 領牌數 1007 ___ 1.4 ___...,Peugeot_308
1258,International Engine Year Awards 2009 BestNewE...,Peugeot_308
1259,Peugeot 歷史 車型 回顧 過往 歷屆 Peugeot 代理 時代 引進 車型 仍然 ...,Peugeot_308
1260,連四歲 喜歡 將於 今年 七月 滿四歲 喜歡 汽車 自然 32 43 汽車 模型 給他 一陣...,Peugeot_308


In [99]:
final_df.type.value_counts()

Peugeot_308    1262
Name: type, dtype: int64

In [100]:
final_df.to_csv('car_nlp_csv//Peugeot_type.csv')

In [2]:
import pandas as pd 
final_df = pd.read_csv('Peugeot_type.csv')

In [3]:
corpus = final_df['content'].tolist()

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
Y = final_df['type'].tolist()

In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y)
# convert integers to dummy variables (one hot encoding)
dummy_y = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [4]:
from sklearn.model_selection import train_test_split
# splitting data into training set and test set. If random_state is set to an integer, the split datasets are fixed.
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.2, random_state=42)


In [5]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
# define model structure
model = Sequential()
model.add(Dense(10, input_dim=1500, activation = 'relu'))
# model.add(Dense(output_dim=10, input_dim=1500, activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(output_dim=3, input_dim=10, activation='softmax'))
model.add(Dense(6, input_dim=10, activation='softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                1010      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 66        
Total params: 1,076
Trainable params: 1,076
Non-trainable params: 0
_________________________________________________________________


In [5]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Sequential
import numpy as np
vocab_size = np.max([np.max(X_train[i]) for i in range(X_train.shape[0])]) + 1  # 这里1 代表空格，其索引被认为是0。
 
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=1500))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()
 


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1500, 64)          72128     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1500, 64)          12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 750, 64)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 750, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 750, 128)          24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 375, 128)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 375, 128)         

In [8]:
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import  BatchNormalization
from keras.layers.embeddings import Embedding
from keras.models import Sequential
import numpy as np
vocab_size = np.max([np.max(X_train[i]) for i in range(X_train.shape[0])]) + 1  # 这里1 代表空格，其索引被认为是0。

model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=1500))
model.add(Convolution1D(256, 3, padding='same'))
model.add(MaxPool1D(3,3,padding='same'))
model.add(Convolution1D(128, 3, padding='same'))
model.add(MaxPool1D(3,3,padding='same'))
model.add(Convolution1D(64, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(BatchNormalization()) # (批)规范化层
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6,activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1500, 300)         338100    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1500, 256)         230656    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 500, 256)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 500, 128)          98432     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 167, 128)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 167, 64)           24640     
_________________________________________________________________
flatten_2 (Flatten)          (None, 10688)            

In [6]:
from keras.optimizers import Adam
# Compile model
EPOCHS = 20
INIT_LR = 1e-3
BS = 100
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [7]:
H = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=100)

Train on 21079 samples, validate on 5270 samples
Epoch 1/20


InternalError:  Blas GEMM launch failed : a.shape=(100, 100), b.shape=(100, 10), m=100, n=10, k=100
	 [[node dense_1/MatMul (defined at C:\Users\Big data\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_795]

Function call stack:
keras_scratch_graph


In [11]:
del model

In [8]:
Y_train.shape

(21079, 6)

In [12]:
X_train.shape

(21079, 1500)

In [13]:
Y_train

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [15]:
X_train

array([[0, 0, 0, ..., 1, 4, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 3, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, encoded_Y, test_size=0.2, random_state=42)
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)

NameError: name 'X' is not defined

In [6]:
y_pred = classifier.predict(X_test)

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
cm

array([[2052,   51,   15,   30,   22,   19],
       [ 114,  731,   29,   30,    4,   10],
       [  71,   27,  186,   12,    4,    4],
       [  60,   34,   11,  807,    1,    4],
       [ 111,   16,    4,    7,  243,   38],
       [ 120,   11,    9,    9,   19,  355]], dtype=int64)

In [10]:
Y_test.shape

(5270,)